# Test initialised fitting and sampling.

1. Explore whether fitting the SkillsGP model and using it as a starting point to fit a new match is faster.
2. Explore whether we can use a MAP-fitted skills vector as the initial sample for SkillsGPMCMC.

## Setup.

In [1]:
import os
import sys

os.chdir("..")

In [5]:
import importlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import squareform, pdist

from src import load, munge
import src.models.gp
import src.stats

In [530]:
matches = load.all_matches_df()
matches_ti9 = matches.loc[matches.league_name == "The International 2019"]
players_mat_ti9 = munge.make_match_players_matrix(
    matches_ti9.radiant_players, matches_ti9.dire_players)
players = src.munge.match_df_to_player_df(matches_ti9)

In [531]:
matches_ti9

,startDate,league_name,radiant_name,dire_name,radiantVictory,radiant_nicknames,dire_nicknames,dire_players,dire_valveId,duration,league_id,radiant_players,radiant_valveId,seriesId,startTimestamp
matchId,,,,,,,,,,,,,,,
4888646940,2019-07-07 03:01:41,The International 2019,BOOM ID,Mineski,True,"[Jhocam, Fibroez, Khezcute, _NinetyHearT_, Ter...","[kpii, ninjaboogie, Moon, raging-_-potato, Nik...","[87012746, 91443418, 113457795, 192914280, 412...",543897,1692,10749,"[152859296, 156328257, 237325411, 301750126, 3...",3785359,345260,1562468501000
4888686136,2019-07-07 03:31:36,The International 2019,Adroit,Team Jinesbrus,False,"[@cmlcmlcml, Mac, Bokerino, Boombacs, ]","[March, Forev, Febby, Gunnar, 23savage]","[3940262, 88933594, 112377459, 126238768, 3755...",7225724,2053,10749,"[100616105, 104512126, 113874152, 127077041, 3...",7098928,345265,1562470296000
4888732113,2019-07-07 04:05:31,The International 2019,Team EVOS,496 Gaming,False,"[Whitemon, Vlaicu, Aville, inYourdreaM, Facehu...","[ĐạtBéBỏng, Banana.Xù, DifferentHeaven, R҉tÈo҉...","[126598297, 129645604, 196931374, 198631761, 2...",4817649,2595,10749,"[136829091, 151835528, 179773256, 181716137, 1...",3262331,345271,1562472331000
4888772755,2019-07-07 04:33:55,The International 2019,RESURGENCE,AMPLFY,False,"[InsidiousC, Deth, keeeeeeeeepo, LJSL, ponlo]","[Jeyo, Karl, ryOyr, Abat.LFT, Carlo]","[87360406, 94281932, 115102590, 119898344, 156...",6797380,3424,10749,"[87442297, 99983413, 109163420, 144870065, 193...",6765426,345278,1562474035000
4888859291,2019-07-07 05:20:41,The International 2019,Team EVOS,Adroit,False,"[Whitemon, Vlaicu, Aville, inYourdreaM, Facehu...","[@cmlcmlcml, Mac, Bokerino, Boombacs, ]","[100616105, 104512126, 113874152, 127077041, 3...",7098928,1954,10749,"[136829091, 151835528, 179773256, 181716137, 1...",3262331,345281,1562476841000
4888936676,2019-07-07 06:01:24,The International 2019,Mineski,AMPLFY,True,"[kpii, ninjaboogie, Moon, raging-_-potato, Nik...","[Jeyo, Karl, ryOyr, Abat.LFT, Carlo]","[87360406, 94281932, 115102590, 119898344, 156...",6797380,1937,10749,"[87012746, 91443418, 113457795, 192914280, 412...",543897,345286,1562479284000
4888974952,2019-07-07 06:22:32,The International 2019,BOOM ID,RESURGENCE,True,"[Jhocam, Fibroez, Khezcute, _NinetyHearT_, Ter...","[InsidiousC, Deth, keeeeeeeeepo, LJSL, ponlo]","[87442297, 99983413, 109163420, 144870065, 193...",6765426,1999,10749,"[152859296, 156328257, 237325411, 301750126, 3...",3785359,345295,1562480552000
4889038294,2019-07-07 06:53:26,The International 2019,Team Jinesbrus,496 Gaming,True,"[March, Forev, Febby, Gunnar, 23savage]","[ĐạtBéBỏng, Banana.Xù, DifferentHeaven, R҉tÈo҉...","[126598297, 129645604, 196931374, 198631761, 2...",4817649,2629,10749,"[3940262, 88933594, 112377459, 126238768, 3755...",7225724,345300,1562482406000
4889115579,2019-07-07 07:36:33,The International 2019,BOOM ID,AMPLFY,False,"[Jhocam, Fibroez, Khezcute, _NinetyHearT_, Ter...","[Jeyo, Karl, ryOyr, Abat.LFT, Carlo]","[87360406, 94281932, 115102590, 119898344, 156...",6797380,2116,10749,"[152859296, 156328257, 237325411, 301750126, 3...",3785359,345311,1562484993000


## First fit a MAP model.

In [532]:
importlib.reload(src.models.gp)

two_years_in_ms = 2 * 365 * 24 * 60 * 60 * 1000
gp_map = src.models.gp.SkillsGPMAP(
    None,
    None,
    players_mat_ti9,
    matches_ti9.startTimestamp.values,
    matches_ti9.radiantVictory,
    "exponential",
    {"scale": two_years_in_ms},
    radi_prior_sd=100,  # Non-informative prior
    logistic_scale=1
)

In [533]:
gp_map.fit()

     fun: -7251.0131626120465
     jac: array([ 1.03590425e-05, -1.17897471e-05, -1.97483157e-06, ...,
        2.08626483e-05, -9.67426324e-06,  1.05673780e-05])
 message: 'Optimization terminated successfully.'
    nfev: 18
    nhev: 2119
     nit: 16
    njev: 33
  status: 0
 success: True
       x: array([-0.28478629, -0.28515649, -0.28527434, ..., -0.16480331,
       -0.16469054, -0.08426577])


In [534]:
gp_map.skills_mat(gp_map.fitted.x[:-1]).iloc[-1].dropna()

6922000      0.596078
10366616     0.596078
84429681     0.305941
86725175     0.305941
86726887     0.596078
87177591     0.305941
87196890     0.305941
94786276     0.305941
108452107    0.596078
221666230    0.596078
Name: 4904311356, dtype: float64

It took us 16 iterations and 2144 Hessian evaluations.

## Fit all but the last match, then add the last match.

In [535]:
matches_to_minus1 = matches_ti9.iloc[:-1]
players_mat_to_minus1 = munge.make_match_players_matrix(
    matches_to_minus1.radiant_players, matches_to_minus1.dire_players)
players_to_minus1 = src.munge.match_df_to_player_df(matches_to_minus1)

matches_minus1 = matches_ti9.iloc[[-1]]
players_mat_minus1 = munge.make_match_players_matrix(
    matches_minus1.radiant_players, matches_minus1.dire_players)
players_minus1 = src.munge.match_df_to_player_df(matches_minus1)

In [536]:
importlib.reload(src.models.gp)

two_years_in_ms = 2 * 365 * 24 * 60 * 60 * 1000
gp_map2 = src.models.gp.SkillsGPMAP(
    None,
    None,
    players_mat_to_minus1,
    matches_to_minus1.startTimestamp.values,
    matches_to_minus1.radiantVictory,
    "exponential",
    {"scale": two_years_in_ms},
    radi_prior_sd=100,
    logistic_scale=1
)

In [537]:
gp_map2.fit()

     fun: -7215.896591841696
     jac: array([-4.42373891e-06,  2.27665835e-06, -6.43913576e-06, ...,
        2.74491658e-06, -6.37987750e-06,  2.63498046e-05])
 message: 'Optimization terminated successfully.'
    nfev: 21
    nhev: 2561
     nit: 18
    njev: 38
  status: 0
 success: True
       x: array([-0.2846862 , -0.28505657, -0.28517459, ..., -0.16446207,
       -0.16434913, -0.07946344])


Idea:

1. Update the players matrix.
2. Check if the players of the new match are already fitted. If so, take the latest value. Otherwise use 0.0.
3. Convert the skills vector into a skills matrix.

### Try `predict()` and `predict_matches()`.

In [538]:
matches_minus1

,startDate,league_name,radiant_name,dire_name,radiantVictory,radiant_nicknames,dire_nicknames,dire_players,dire_valveId,duration,league_id,radiant_players,radiant_valveId,seriesId,startTimestamp
matchId,,,,,,,,,,,,,,,
4904311356,2019-07-15 01:30:53,The International 2019,J.Storm,Forward Gaming,False,"[Moo, Resolut1on, Fear, 1437, nine]","[pieliedie, Sneyking, MSS, YawaR, CCnC]","[6922000, 10366616, 86726887, 108452107, 22166...",6214538,1964,10749,"[84429681, 86725175, 87177591, 87196890, 94786...",6288801,348164,1563154253000


In [539]:
matches_minus1.dire_players.iloc[0] + matches_minus1.radiant_players.iloc[0]

[6922000,
 10366616,
 86726887,
 108452107,
 221666230,
 84429681,
 86725175,
 87177591,
 87196890,
 94786276]

In [540]:
list(zip((matches_to_minus1.radiant_players + matches_to_minus1.dire_players),
          matches_to_minus1.startTimestamp))[:2]

[([152859296,
   156328257,
   237325411,
   301750126,
   389033587,
   87012746,
   91443418,
   113457795,
   192914280,
   412753955],
  1562468501000),
 ([100616105,
   104512126,
   113874152,
   127077041,
   355168766,
   3940262,
   88933594,
   112377459,
   126238768,
   375507918],
  1562470296000)]

In [541]:
pred_mat, var_mat = gp_map2.predict(
    [1562468501000],
    (matches_minus1.radiant_players.iloc[0]
     + matches_minus1.dire_players.iloc[0]))
display(pred_mat)
var_mat

,84429681,86725175,87177591,87196890,94786276,6922000,10366616,86726887,108452107,221666230
1562468501000,0.312868,0.312868,0.312868,0.312868,0.312868,0.575613,0.575613,0.575613,0.575613,0.575613


,84429681,86725175,87177591,87196890,94786276,6922000,10366616,86726887,108452107,221666230
1562468501000,0.012488,0.012488,0.012488,0.012488,0.012488,0.012488,0.012488,0.012488,0.012488,0.012488


In [542]:
display(matches_minus1)
gp_map2.predict_matches(matches_minus1.radiant_players,
                        matches_minus1.dire_players,
                        matches_minus1.startTimestamp.values)

,startDate,league_name,radiant_name,dire_name,radiantVictory,radiant_nicknames,dire_nicknames,dire_players,dire_valveId,duration,league_id,radiant_players,radiant_valveId,seriesId,startTimestamp
matchId,,,,,,,,,,,,,,,
4904311356,2019-07-15 01:30:53,The International 2019,J.Storm,Forward Gaming,False,"[Moo, Resolut1on, Fear, 1437, nine]","[pieliedie, Sneyking, MSS, YawaR, CCnC]","[6922000, 10366616, 86726887, 108452107, 22166...",6214538,1964,10749,"[84429681, 86725175, 87177591, 87196890, 94786...",6288801,348164,1563154253000


,pred_win_prob,win_prob-2sd,win_prob+2sd,radi_skill,radi_skill_sd,dire_skill,dire_skill_sd,radi_adv
1563154253000,0.200174,0.188626,0.212244,1.578399,0.026076,2.884142,0.026076,-0.079463


### Manually create a new SkillsGPMAP object.

In [543]:
fitted_skills_mat = gp_map2.skills_mat(gp_map2.fitted.x[:-1])
fitted_skills_mat

,3916428,3940262,4281729,6922000,10366616,11550182,12231202,18180970,20321748,26316691,...,352545711,355168766,375507918,389033587,407321629,412753955,412758827,468599935,480412663,893135754
matchId,,,,,,,,,,,,,,,,,,,,,
4888646940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-0.036837,NaN,0.208715,NaN,NaN,NaN,NaN
4888686136,NaN,0.197878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.069679,0.197878,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4888732113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4888772755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4888859291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.069755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4888936676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.209019,NaN,NaN,NaN,NaN
4888974952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-0.037145,NaN,NaN,NaN,NaN,NaN,NaN
4889038294,NaN,0.197790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.197790,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4889115579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-0.037293,NaN,NaN,NaN,NaN,NaN,NaN


In [544]:
def last_value_or_0(skills_mat, col_idx, default=0.0):
    """Get the last valid value or the default value in series s."""
    if col_idx not in skills_mat.columns:
        return default
    else:
        s = skills_mat[col_idx]
        idx = s.last_valid_index()
        if idx is None:
            return default
        else:
            return s[idx]

In [545]:
fitted_skills_mat.tail()

,3916428,3940262,4281729,6922000,10366616,11550182,12231202,18180970,20321748,26316691,...,352545711,355168766,375507918,389033587,407321629,412753955,412758827,468599935,480412663,893135754
matchId,,,,,,,,,,,,,,,,,,,,,
4903974414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4904062522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4904171017,NaN,NaN,NaN,0.576858,0.576858,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4904218149,NaN,NaN,NaN,0.576875,0.576875,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4904261164,NaN,NaN,NaN,0.576868,0.576868,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [546]:
imputed_skills = pd.Series([last_value_or_0(fitted_skills_mat, i)
                            for i in players_mat_minus1.columns],
                           index=players_mat_minus1.columns,
                           name=players_mat_minus1.index[0])
pd.DataFrame(imputed_skills).T

,6922000,10366616,84429681,86725175,86726887,87177591,87196890,94786276,108452107,221666230
4904311356,0.576868,0.576868,0.315701,0.315701,0.576868,0.315701,0.315701,0.315701,0.576868,0.576868


In [547]:
new_skills_mat = fitted_skills_mat.append(pd.DataFrame(imputed_skills).T)
new_skills_mat.tail()

,3916428,3940262,4281729,6922000,10366616,11550182,12231202,18180970,20321748,26316691,...,352545711,355168766,375507918,389033587,407321629,412753955,412758827,468599935,480412663,893135754
4904062522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4904171017,NaN,NaN,NaN,0.576858,0.576858,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4904218149,NaN,NaN,NaN,0.576875,0.576875,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4904261164,NaN,NaN,NaN,0.576868,0.576868,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4904311356,NaN,NaN,NaN,0.576868,0.576868,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [548]:
players_mat_minus1

,6922000,10366616,84429681,86725175,86726887,87177591,87196890,94786276,108452107,221666230
matchId,,,,,,,,,,
4904311356,-1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0


In [549]:
new_players_mat = gp_map2.players_mat.append(players_mat_minus1).fillna(0.0)
new_players_mat

,3916428,3940262,4281729,6922000,10366616,11550182,12231202,18180970,20321748,26316691,...,352545711,355168766,375507918,389033587,407321629,412753955,412758827,468599935,480412663,893135754
matchId,,,,,,,,,,,,,,,,,,,,,
4888646940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0
4888686136,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4888732113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4888772755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4888859291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4888936676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4888974952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4889038294,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4889115579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### Actually create the new object with the computed values.

In [550]:
importlib.reload(src.models.gp)

new_initial_skills = new_skills_mat.values.T[new_players_mat.values.T != 0.0]

two_years_in_ms = 2 * 365 * 24 * 60 * 60 * 1000
gp_map2_prime = src.models.gp.SkillsGPMAP(
    new_initial_skills,
    gp_map2.fitted.x[-1],
    new_players_mat,
    matches_ti9.startTimestamp.values,
    matches_ti9.radiantVictory,
    "exponential",
    {"scale": two_years_in_ms},
    radi_prior_sd=100,
    logistic_scale=1
)

In [551]:
gp_map2_prime.fit()

     fun: -7251.013162612773
     jac: array([ 6.18084507e-06, -8.36016304e-06, -5.12338929e-06, ...,
       -8.23186910e-06, -2.32293850e-06, -5.05975558e-05])
 message: 'Optimization terminated successfully.'
    nfev: 9
    nhev: 1163
     nit: 8
    njev: 16
  status: 0
 success: True
       x: array([-0.28478576, -0.28515596, -0.28527381, ..., -0.16480323,
       -0.16469046, -0.08426584])


By imputing the skills of the new samples, we get the following speedup:

* Time: from 1160 ms to 410 ms (65% decrease).
* Iterations: from 16 to 5 (68% decrease).
* Hessian evaluations: from 2144 to 678 (68% decrease).
* Objective: from -7251.013162611372 to -7251.013158887112.

In [552]:
gp_map2_prime.skills_mat(gp_map2_prime.fitted.x[:-1]).iloc[-1].dropna()

6922000      0.596078
10366616     0.596078
84429681     0.305941
86725175     0.305941
86726887     0.596078
87177591     0.305941
87196890     0.305941
94786276     0.305941
108452107    0.596078
221666230    0.596078
Name: 4904311356, dtype: float64

For references, the results from the full fitting are here.

    87012746     0.208980
    91443418     0.208980
    113457795    0.208980
    152859296   -0.036104
    156328257   -0.036104
    192914280    0.208980
    237325411   -0.036104
    301750126   -0.036104
    389033587   -0.036104
    412753955    0.208980
    Name: 4888646940, dtype: float64

In [553]:
gp_map2_prime.win_prob(0.005)

0.5012499973958399

### Incremental fitting based on `SkillsGPMAP.add_matches()`.

In [562]:
matches_minus1

,startDate,league_name,radiant_name,dire_name,radiantVictory,radiant_nicknames,dire_nicknames,dire_players,dire_valveId,duration,league_id,radiant_players,radiant_valveId,seriesId,startTimestamp
matchId,,,,,,,,,,,,,,,
4904311356,2019-07-15 01:30:53,The International 2019,J.Storm,Forward Gaming,False,"[Moo, Resolut1on, Fear, 1437, nine]","[pieliedie, Sneyking, MSS, YawaR, CCnC]","[6922000, 10366616, 86726887, 108452107, 22166...",6214538,1964,10749,"[84429681, 86725175, 87177591, 87196890, 94786...",6288801,348164,1563154253000


In [563]:
importlib.reload(src.models.gp)

two_years_in_ms = 2 * 365 * 24 * 60 * 60 * 1000
gp_map2 = src.models.gp.SkillsGPMAP(
    None,
    None,
    players_mat_to_minus1,
    matches_to_minus1.startTimestamp.values,
    matches_to_minus1.radiantVictory,
    "exponential",
    {"scale": two_years_in_ms},
    radi_prior_sd=100,
    logistic_scale=1
)

In [564]:
gp_map2.fit()

     fun: -7215.896591841696
     jac: array([-4.42373891e-06,  2.27665835e-06, -6.43913576e-06, ...,
        2.74491658e-06, -6.37987750e-06,  2.63498046e-05])
 message: 'Optimization terminated successfully.'
    nfev: 21
    nhev: 2561
     nit: 18
    njev: 38
  status: 0
 success: True
       x: array([-0.2846862 , -0.28505657, -0.28517459, ..., -0.16446207,
       -0.16434913, -0.07946344])


In [565]:
gp_map2_prime = gp_map2.add_matches(players_mat_minus1,
                                    matches_minus1.startTimestamp,
                                    matches_minus1.radiantVictory)

In [566]:
gp_map2_prime.fit()

     fun: -7251.013162612773
     jac: array([ 6.18084507e-06, -8.36016304e-06, -5.12338929e-06, ...,
       -8.23186910e-06, -2.32293850e-06, -5.05975558e-05])
 message: 'Optimization terminated successfully.'
    nfev: 9
    nhev: 1163
     nit: 8
    njev: 16
  status: 0
 success: True
       x: array([-0.28478576, -0.28515596, -0.28527381, ..., -0.16480323,
       -0.16469046, -0.08426584])


### Compare the fitting results from obtained from full vs incremental fit.

In [574]:
gp_map.skills_mat(gp_map.fitted.x[:-1]).iloc[-1].dropna()

6922000      0.596078
10366616     0.596078
84429681     0.305941
86725175     0.305941
86726887     0.596078
87177591     0.305941
87196890     0.305941
94786276     0.305941
108452107    0.596078
221666230    0.596078
Name: 4904311356, dtype: float64

In [575]:
gp_map2_prime.skills_mat(gp_map2_prime.fitted.x[:-1]).iloc[-1].dropna()

6922000      0.596078
10366616     0.596078
84429681     0.305941
86725175     0.305941
86726887     0.596078
87177591     0.305941
87196890     0.305941
94786276     0.305941
108452107    0.596078
221666230    0.596078
Name: 4904311356, dtype: float64

## Test incremental fitting, but now using the full dataset.

In [ ]:
players_mat = munge.make_match_players_matrix(
    matches.radiant_players, matches.dire_players)
players = src.munge.match_df_to_player_df(matches)

In [ ]:
matches_to_minus1 = matches.iloc[:-1]
players_mat_to_minus1 = munge.make_match_players_matrix(
    matches_to_minus1.radiant_players, matches_to_minus1.dire_players)
players_to_minus1 = src.munge.match_df_to_player_df(matches_to_minus1)

matches_minus1 = matches.iloc[[-1]]
players_mat_minus1 = munge.make_match_players_matrix(
    matches_minus1.radiant_players, matches_minus1.dire_players)
players_minus1 = src.munge.match_df_to_player_df(matches_minus1)

In [ ]:
importlib.reload(src.models.gp)

two_years_in_ms = 2 * 365 * 24 * 60 * 60 * 1000
gp_map3 = src.models.gp.SkillsGPMAP(
    None,
    None,
    players_mat_to_minus1,
    matches_to_minus1.startTimestamp.values,
    matches_to_minus1.radiantVictory,
    players.loc[players_mat_to_minus1.columns].name,
    "exponential",
    {"scale": two_years_in_ms},
    radi_prior_sd=100,
    logistic_scale=1
)

In [ ]:
gp_map3.fit()

### Now perform the incremental fit.

In [84]:
matches.iloc[-1]

dire_name                                                   Rock.Young
dire_nicknames                        [Jixing, Gintoki, Ayo, 垃圾君, Lin]
dire_players         [92590479, 101908305, 110228930, 116852311, 13...
dire_valveId                                                   4288603
duration                                                          1566
league_id                                                         9601
league_name                                               EPICENTER XL
radiantVictory                                                    True
radiant_name                                               VGJ Thunder
radiant_nicknames                     [Sylar, ddc, Freeze, Yang, Fade]
radiant_players      [108382060, 114239371, 137272985, 139937922, 1...
radiant_valveId                                                5027210
seriesId                                                        209169
startDate                                          2018-03-23 08:10:59
startT

In [89]:
fitted_skills_mat = gp_map3.skills_mat(gp_map3.fitted.x[:-1])
imputed_skills = pd.Series([last_value_or_0(fitted_skills_mat, i)
                            for i in players_mat_minus1.columns],
                           index=players_mat_minus1.columns,
                           name=players_mat_minus1.index[0])
pd.DataFrame(imputed_skills).T

,92590479,101908305,108382060,110228930,114239371,116852311,136477860,137272985,139937922,182331313
3795410349,-0.158531,0.242847,0.187159,0.28641,0.192424,0.242847,0.369735,-0.041949,0.214477,0.742586


In [90]:
new_skills_mat = fitted_skills_mat.append(pd.DataFrame(imputed_skills).T)

new_players_mat = gp_map3.players_mat.append(players_mat_minus1).fillna(0.0)

In [92]:
importlib.reload(src.models.gp)

new_initial_skills = new_skills_mat.values.T[new_players_mat.values.T != 0.0]

two_years_in_ms = 2 * 365 * 24 * 60 * 60 * 1000
gp_map3_prime = src.models.gp.SkillsGPMAP(
    new_initial_skills,
    gp_map3.fitted.x[-1],
    new_players_mat,
    matches.startTimestamp.values,
    matches.radiantVictory,
    players.loc[players_mat.columns].name,
    "exponential",
    {"scale": two_years_in_ms},
    radi_prior_sd=100,
    logistic_scale=1
)

In [93]:
gp_map3_prime.fit()

     fun: -140676.34710956446
     jac: array([ 3.50886888e-06,  2.86890205e-06, -7.95669708e-06, ...,
       -1.57840128e-06,  3.07009357e-07,  8.79081164e-05])
 message: 'Optimization terminated successfully.'
    nfev: 5
    nhev: 5198
     nit: 4
    njev: 8
  status: 0
 success: True
       x: array([0.74687263, 0.74696523, 0.74989353, ..., 0.10355485, 0.10417335,
       0.17807179])
